In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Partitioning_Exercises") \
    .getOrCreate()

In [ ]:
data = [
    ("O001","Hyderabad","Electronics",1200,"Delivered"),
    ("O002","Delhi","Clothing",800,"Delivered"),
    ("O003","Mumbai","Electronics",1500,"Cancelled"),
    ("O004","Bangalore","Grocery",400,"Delivered"),
    ("O005","Hyderabad","Grocery",300,"Delivered"),
    ("O006","Delhi","Electronics",2000,"Delivered"),
    ("O007","Mumbai","Clothing",700,"Delivered"),
    ("O008","Bangalore","Electronics",1800,"Delivered"),
    ("O009","Delhi","Grocery",350,"Cancelled"),
    ("O010","Hyderabad","Clothing",900,"Delivered")
]

columns=["order_id","city","category","order_amount","status"]

df= spark.createDataFrame(data, columns)
df.show()
df.printSchema()


+--------+---------+-----------+------------+---------+
|order_id|     city|   category|order_amount|   status|
+--------+---------+-----------+------------+---------+
|    O001|Hyderabad|Electronics|        1200|Delivered|
|    O002|    Delhi|   Clothing|         800|Delivered|
|    O003|   Mumbai|Electronics|        1500|Cancelled|
|    O004|Bangalore|    Grocery|         400|Delivered|
|    O005|Hyderabad|    Grocery|         300|Delivered|
|    O006|    Delhi|Electronics|        2000|Delivered|
|    O007|   Mumbai|   Clothing|         700|Delivered|
|    O008|Bangalore|Electronics|        1800|Delivered|
|    O009|    Delhi|    Grocery|         350|Cancelled|
|    O010|Hyderabad|   Clothing|         900|Delivered|
+--------+---------+-----------+------------+---------+

root
 |-- order_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- order_amount: long (nullable = true)
 |-- status: string (nullable = true)



In [ ]:
df.rdd.getNumPartitions()

2

In [ ]:
df_repart= df.repartition(4)
df_repart.rdd.getNumPartitions()

4

In [ ]:
df_coalesce = df_repart.coalesce(1)
df_coalesce.rdd.getNumPartitions()

1

✅ repartition() vs coalesce() in Spark
1. repartition(numPartitions)

What it does:
Creates a new DataFrame with exactly numPartitions partitions.
How:
It shuffles the data across the cluster to evenly distribute rows among the specified number of partitions.
When to use:

When you increase the number of partitions (e.g., from 4 to 10).
When you want better parallelism for large operations like joins or aggregations.


Cost:
Expensive because it involves a full shuffle of data.


2. coalesce(numPartitions)

What it does:
Reduces the number of partitions without a full shuffle.
How:
It merges existing partitions into fewer partitions.
When to use:

When you decrease the number of partitions (e.g., from 10 to 4).
When you want to avoid unnecessary shuffling.


Cost:
Much cheaper than repartition() because it avoids a full shuffle.


✅ Performance Impact

Too few partitions:
Tasks become large → less parallelism → slower execution.
Too many partitions:
Overhead in task scheduling → small tasks → inefficient.
Rule of thumb:
Aim for 128 MB per partition for optimal performance.

In [ ]:
#This is the Transformation part
filtered_df = df.filter(df.city=="Delhi")
selected_df = filtered_df.select("order_id","order_amount")

In [ ]:
#This is the Action Part
selected_df.show()

+--------+------------+
|order_id|order_amount|
+--------+------------+
|    O002|         800|
|    O006|        2000|
|    O009|         350|
+--------+------------+



In [ ]:
#This is the transformation part: where it is transformed but won't be printed until its acted upon
df_lineage = (
    df.filter(df.status=="Delivered")
    .filter(df.order_amount>500)
    .select("city","order_amount")
)

In [ ]:
#This is action part that prints the part it remembered in the transformation part in a way we want
df_lineage.count()

6

In [ ]:
df.explain(True)

== Parsed Logical Plan ==
LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Analyzed Logical Plan ==
order_id: string, city: string, category: string, order_amount: bigint, status: string
LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Optimized Logical Plan ==
LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Physical Plan ==
*(1) Scan ExistingRDD[order_id#0,city#1,category#2,order_amount#3L,status#4]



Explain shows the 4 stages through which the data passes through Spark





In [ ]:
df_lineage.explain(True)

== Parsed Logical Plan ==
'Project ['city, 'order_amount]
+- Filter (order_amount#3L > cast(500 as bigint))
   +- Filter (status#4 = Delivered)
      +- LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Analyzed Logical Plan ==
city: string, order_amount: bigint
Project [city#1, order_amount#3L]
+- Filter (order_amount#3L > cast(500 as bigint))
   +- Filter (status#4 = Delivered)
      +- LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Optimized Logical Plan ==
Project [city#1, order_amount#3L]
+- Filter ((isnotnull(status#4) AND isnotnull(order_amount#3L)) AND ((status#4 = Delivered) AND (order_amount#3L > 500)))
   +- LogicalRDD [order_id#0, city#1, category#2, order_amount#3L, status#4], false

== Physical Plan ==
*(1) Project [city#1, order_amount#3L]
+- *(1) Filter ((isnotnull(status#4) AND isnotnull(order_amount#3L)) AND ((status#4 = Delivered) AND (order_amount#3L > 500)))
   +- *(1) Scan ExistingRDD[order_id#0,ci

For lineage df it shows more plans as the data is selected twice and filtered before its output is shown